Script to do analysis on the integrated data (FAIRE + RNA-seq), chi-squared test. Change the overlap and min_reps parameters to get the combination that you want. Default here: overlap = '0.9', minreps = 1 

In [5]:
import os
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

In [6]:
# minimum number of peaks in each condition (here male/partheno)
min_reps = '1'
# overlap used with bedtools intersect (-F parameter)
overlap = '0.9'  # max = 1.0

In [7]:
cwd = os.getcwd() # current working directory. All scripts in this directory. Inside : RNA, FAIRE, and integrated
RNA_directory = cwd + '/RNA/' # there's also the R scripts and result of the AskoR analysis inside
FAIRE_directory = cwd + '/FAIRE/'
integrated_directory = cwd + '/integrated/'

In [8]:
# Prepare all headers for the file imports
header_DAR_broad = ['chr', 'start', 'end', 'interval_id', 'num_peaks', 'num_samples', 'Male1_bool', 'Male2_bool', 'Partheno1_bool', 'Partheno2_bool', 'Partheno3_bool', 'Male1_fc', 'Male2_fc', 'Partheno1_fc', 'Partheno2_fc', 'Partheno3_fc', 'Male1_qval', 'Male2_qval', 'Partheno1_qval', 'Partheno2_qval', 'Partheno3_qval', 'Male1_pval', 'Male2_pval', 'Partheno1_pval', 'Partheno2_pval', 'Partheno3_pval', 'Male1_start', 'Male2_start', 'Partheno1_start', 'Partheno2_start', 'Partheno3_start', 'Male1_end', 'Male2_end', 'Partheno1_end', 'Partheno2_end', 'Partheno3_end']
header_DAR_narrow = header_DAR_broad + ['Male1_summit', 'Male2_summit', 'Partheno1_summit', 'Partheno2_summit', 'Partheno3_summit']
header_DEG_gff = ['seqid', 'source', 'type', 'DEG_start', 'DEG_end', 'score', 'strand', 'phase', 'attributes']
header_DEG_bed = ['seqid', 'DEG_start', 'DEG_end', 'strand', 'gene']
header_DAR_broad_DEG = header_DAR_broad[:4] + ['strand', 'gene'] + header_DAR_broad[4:]
header_DAR_narrow_DEG = header_DAR_narrow[:4] + ['strand', 'gene'] + header_DAR_narrow[4:]

In [9]:
# DAR broad
DAR_broad_male_path = f'{FAIRE_directory}DAR_broad_male_minreps{min_reps}.bed'
DAR_broad_partheno_path = f'{FAIRE_directory}DAR_broad_partheno_minreps{min_reps}.bed'
intergroup_broad_path = f'{FAIRE_directory}DAR_broad_intergroup_minreps{min_reps}.bed'
DAR_broad_male = pd.read_csv(DAR_broad_male_path, sep='\t', names=header_DAR_broad)
DAR_broad_partheno = pd.read_csv(DAR_broad_partheno_path, sep='\t', names=header_DAR_broad)
intergroup_broad = pd.read_csv(intergroup_broad_path, sep='\t', names=header_DAR_broad)

# DAR narrow
DAR_narrow_male_path = f'{FAIRE_directory}DAR_narrow_male_minreps{min_reps}.bed'
DAR_narrow_partheno_path = f'{FAIRE_directory}DAR_narrow_partheno_minreps{min_reps}.bed'
intergroup_narrow_path = f'{FAIRE_directory}DAR_narrow_intergroup_minreps{min_reps}.bed'
DAR_narrow_male = pd.read_csv(DAR_narrow_male_path, sep='\t', names=header_DAR_narrow)
DAR_narrow_partheno = pd.read_csv(DAR_narrow_partheno_path, sep='\t', names=header_DAR_narrow)
intergroup_narrow = pd.read_csv(intergroup_narrow_path, sep='\t', names=header_DAR_narrow)

DAR_broad_male

,chr,start,end,interval_id,num_peaks,num_samples,Male1_bool,Male2_bool,Partheno1_bool,Partheno2_bool,...,Male1_start,Male2_start,Partheno1_start,Partheno2_start,Partheno3_start,Male1_end,Male2_end,Partheno1_end,Partheno2_end,Partheno3_end
0,scaffold_1,43510,43769,Interval_2,2,2,True,True,False,False,...,43510,43516.0,NaN,NaN,NaN,43698,43769.0,NaN,NaN,NaN
1,scaffold_1,205473,205653,Interval_8,1,1,True,False,False,False,...,205473,NaN,NaN,NaN,NaN,205653,NaN,NaN,NaN,NaN
2,scaffold_1,666042,666280,Interval_25,1,1,False,True,False,False,...,NaN,666042.0,NaN,NaN,NaN,NaN,666280.0,NaN,NaN,NaN
3,scaffold_1,856129,856490,Interval_37,2,2,True,True,False,False,...,856129,856200.0,NaN,NaN,NaN,856490,856409.0,NaN,NaN,NaN
4,scaffold_1,1054779,1055762,Interval_46,1,1,False,True,False,False,...,NaN,1054779.0,NaN,NaN,NaN,NaN,1055762.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14502,scaffold_553,3414,3767,Interval_44207,1,1,False,True,False,False,...,NaN,3414.0,NaN,NaN,NaN,NaN,3767.0,NaN,NaN,NaN
14503,scaffold_75,21132,21352,Interval_44231,1,1,False,True,False,False,...,NaN,21132.0,NaN,NaN,NaN,NaN,21352.0,NaN,NaN,NaN
14504,scaffold_81,300,688,Interval_44238,1,1,False,True,False,False,...,NaN,300.0,NaN,NaN,NaN,NaN,688.0,NaN,NaN,NaN
14505,scaffold_88,18958,19913,Interval_44245,2,2,True,True,False,False,...,19225,18958.0,NaN,NaN,NaN,19454,19913.0,NaN,NaN,NaN


In [10]:
# DEG
DEG_male_tss_bed = pd.read_csv(RNA_directory + 'DEG_male_tss.bed', sep='\t', names=header_DEG_bed)
DEG_partheno_tss_bed = pd.read_csv(RNA_directory + 'DEG_partheno_tss.bed', sep='\t', names=header_DEG_bed)

DEG_male_tss_bed

,seqid,DEG_start,DEG_end,strand,gene
0,scaffold_1,182836,185836,+,g168
1,scaffold_1,222426,225426,-,g169
2,scaffold_1,223016,226016,+,g170
3,scaffold_1,614736,617736,-,g190
4,scaffold_1,614956,617956,+,g191
...,...,...,...,...,...
2862,scaffold_418,0,1501,+,g33
2863,scaffold_462,3046,6046,-,g31176
2864,scaffold_488,0,1501,+,g20198
2865,scaffold_512,1576,4576,-,g22836


In [13]:
# all genes X all expressed to get only all the expressed genes, but keeping the columns of the gff all genes table
all_genes = pd.read_csv('Acyrthosiphon_pisum_JIC1_v1.0.scaffolds.braker2.gff', sep='\t', names=header_DEG_gff)
all_genes = all_genes[all_genes['type'] == 'gene']
# Adding a column gene to the gff to make the join with the DEG lists easier
all_genes['gene'] = all_genes['attributes'].str.extract(r'ID=(.*);')
# Creating a new bed file with -/+ 1500pb around the TSS to define a larger promoter region
all_genes_tss = pd.DataFrame({
    'seqid': all_genes['seqid'],
    'DEG_start': all_genes.apply(lambda x: x['DEG_start'] - 1500 if x['strand'] == '+' else x['DEG_end'] - 1500, axis=1).clip(lower=0),
    'DEG_end': all_genes.apply(lambda x: x['DEG_start'] + 1500 if x['strand'] == '+' else x['DEG_end'] + 1500, axis=1).clip(lower=0),
    'strand': all_genes['strand'],
    'gene': all_genes['gene']
})
all_expressed_path = RNA_directory + 'AskoR/MaleVsPartheno/DEanalysis/DEtables/MalevsPartheno.txt'
all_expressed = pd.read_csv(all_expressed_path, sep='\t')
all_expressed

,gene,contrast,Expression,Significance,PValue,logFC,FC,FDR,Male_NormMeanCount,Partheno_NormMeanCount,Male_CPMnormMeanCount,Partheno_CPMnormMeanCount,Description
0,g1040,MalevsPartheno,Male>Partheno,1,6.416811e-10,10.655157,1612.581797,3.717083e-07,1855.157830,1.028444,123.374449,0.068395,B-like cysteine ase 3
1,g1052,MalevsPartheno,Male>Partheno,1,3.633853e-10,10.739071,1709.158793,3.717083e-07,17499.549755,10.046700,1163.780935,0.668141,B-like cysteine ase 3
2,g13894,MalevsPartheno,Male>Partheno,1,3.030168e-10,11.579582,3060.564620,3.717083e-07,4497.057113,1.369622,299.069942,0.091085,SUMO- ligase PIAS2-like
3,g13901,MalevsPartheno,Male>Partheno,1,3.285241e-10,10.223023,1195.188721,3.717083e-07,4413.650911,3.534850,293.523140,0.235080,-N-acetylglucosamine--peptide N-acetylglucosam...
4,g15240,MalevsPartheno,Male>Partheno,1,9.454212e-10,10.325433,1283.113518,3.717083e-07,1880.041721,1.365800,125.029315,0.090830,facilitator superfamily domain-containing 6 B
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14103,g9983,MalevsPartheno,Male=Partheno,0,9.732021e-01,0.338480,1.264423,9.999986e-01,1897.972198,1501.029867,126.221750,99.823710,NaN
14104,g9984,MalevsPartheno,Male=Partheno,0,9.981760e-01,0.025302,1.017693,9.999986e-01,1913.176933,1879.553424,127.232900,124.996800,catecholamines up-like
14105,g9985,MalevsPartheno,Male=Partheno,0,7.550673e-01,-0.593703,1.509115,9.999986e-01,143.294916,217.053953,9.529610,14.434840,ST7 homolog
14106,g9986,MalevsPartheno,Male=Partheno,0,6.892120e-01,0.745883,1.677001,9.999986e-01,3310.595731,1974.267320,220.166100,131.295600,pellucida domain


In [14]:
# Keeping only the lines in the gff where there are the expressed genes
all_expressed = all_genes_tss[(all_genes_tss['gene'].isin(all_expressed.gene))]
# used for the run_intersects_TSS.sh
all_expressed.to_csv('Acyrthosiphon_pisum_JIC1_v1.0_expressed_tss.bed', sep='\t', index=False, na_rep='NA', header=False)
all_expressed

,seqid,DEG_start,DEG_end,strand,gene
57,scaffold_1,70696,73696,+,g157
248,scaffold_1,151776,154776,+,g166
304,scaffold_1,182836,185836,+,g168
338,scaffold_1,222426,225426,-,g169
359,scaffold_1,223016,226016,+,g170
...,...,...,...,...,...
892413,scaffold_508,2056,5056,-,g20117
892434,scaffold_512,1576,4576,-,g22836
892464,scaffold_517,0,2266,+,g20130
892584,scaffold_527,1816,4816,+,g22813


In [15]:
# DAR + DEG broad
DAR_broad_DEG_male_path = f'{integrated_directory}DAR_broad_DEG_male_minreps{min_reps}_overlap{overlap}.bed'
DAR_broad_DEG_partheno_path = f'{integrated_directory}DAR_broad_DEG_partheno_minreps{min_reps}_overlap{overlap}.bed'
DAR_broad_DEG_male = pd.read_csv(DAR_broad_DEG_male_path, sep='\t', names=header_DAR_broad_DEG)
DAR_broad_DEG_partheno = pd.read_csv(DAR_broad_DEG_partheno_path, sep='\t', names=header_DAR_broad_DEG)

# DAR + DEG narrow
DAR_narrow_DEG_male_path = f'{integrated_directory}DAR_narrow_DEG_male_minreps{min_reps}_overlap{overlap}.bed'
DAR_narrow_DEG_partheno_path = f'{integrated_directory}DAR_narrow_DEG_partheno_minreps{min_reps}_overlap{overlap}.bed'
DAR_narrow_DEG_male = pd.read_csv(DAR_narrow_DEG_male_path, sep='\t', names=header_DAR_narrow_DEG)
DAR_narrow_DEG_partheno = pd.read_csv(DAR_narrow_DEG_partheno_path, sep='\t', names=header_DAR_narrow_DEG)
DAR_narrow_DEG_partheno

,chr,start,end,interval_id,strand,gene,num_peaks,num_samples,Male1_bool,Male2_bool,...,Male1_end,Male2_end,Partheno1_end,Partheno2_end,Partheno3_end,Male1_summit,Male2_summit,Partheno1_summit,Partheno2_summit,Partheno3_summit
0,scaffold_1,347008,347311,Interval_13,+,g175,1,1,False,False,...,NaN,NaN,347311.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,scaffold_1,442462,442861,Interval_16,+,g180,2,2,False,False,...,NaN,NaN,442861.0,442789,NaN,NaN,NaN,NaN,NaN,NaN
2,scaffold_1,442184,442434,Interval_15,+,g180,1,1,False,False,...,NaN,NaN,NaN,NaN,442434.0,NaN,NaN,NaN,NaN,NaN
3,scaffold_1,2104718,2104930,Interval_58,+,g263,1,1,False,False,...,NaN,NaN,2104930.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,scaffold_1,10908075,10908268,Interval_224,-,g808,1,1,False,False,...,NaN,NaN,10908268.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,scaffold_4,19402907,19403140,Interval_36773,-,g21409,1,1,False,False,...,NaN,NaN,NaN,NaN,19403140.0,NaN,NaN,NaN,NaN,NaN
108,scaffold_4,19402907,19403140,Interval_36773,+,g21410,1,1,False,False,...,NaN,NaN,NaN,NaN,19403140.0,NaN,NaN,NaN,NaN,NaN
109,scaffold_4,32110557,32110772,Interval_38038,+,g22048,1,1,False,False,...,NaN,NaN,32110772.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,scaffold_4,34674158,34674329,Interval_38258,+,g22202,1,1,False,False,...,NaN,NaN,NaN,NaN,34674329.0,NaN,NaN,NaN,NaN,NaN


In [16]:
# DAR in TSS regions (broad)
DAR_broad_TSS_male_path = f'{integrated_directory}DAR_broad_TSS_male_minreps{min_reps}_overlap{overlap}.bed'
DAR_broad_TSS_partheno_path = f'{integrated_directory}DAR_broad_TSS_partheno_minreps{min_reps}_overlap{overlap}.bed'
DAR_broad_TSS_male = pd.read_csv(DAR_broad_TSS_male_path, sep='\t', names=header_DAR_broad_DEG)
DAR_broad_TSS_partheno = pd.read_csv(DAR_broad_TSS_partheno_path, sep='\t', names=header_DAR_broad_DEG)

# DAR in TSS regions (narrow)
DAR_narrow_TSS_male_path = f'{integrated_directory}DAR_narrow_TSS_male_minreps{min_reps}_overlap{overlap}.bed'
DAR_narrow_TSS_partheno_path = f'{integrated_directory}DAR_narrow_TSS_partheno_minreps{min_reps}_overlap{overlap}.bed'
DAR_narrow_TSS_male = pd.read_csv(DAR_narrow_TSS_male_path, sep='\t', names=header_DAR_narrow_DEG)
DAR_narrow_TSS_partheno = pd.read_csv(DAR_narrow_TSS_partheno_path, sep='\t', names=header_DAR_narrow_DEG)

DAR_broad_TSS_male

,chr,start,end,interval_id,strand,gene,num_peaks,num_samples,Male1_bool,Male2_bool,...,Male1_start,Male2_start,Partheno1_start,Partheno2_start,Partheno3_start,Male1_end,Male2_end,Partheno1_end,Partheno2_end,Partheno3_end
0,scaffold_1,1054779,1055762,Interval_46,-,g207,1,1,False,True,...,NaN,1054779.0,NaN,NaN,NaN,NaN,1055762.0,NaN,NaN,NaN
1,scaffold_1,2447629,2447933,Interval_96,-,g283,1,1,False,True,...,NaN,2447629.0,NaN,NaN,NaN,NaN,2447933.0,NaN,NaN,NaN
2,scaffold_1,8508176,8508404,Interval_269,-,g692,1,1,False,True,...,NaN,8508176.0,NaN,NaN,NaN,NaN,8508404.0,NaN,NaN,NaN
3,scaffold_1,8512688,8513408,Interval_270,+,g693,1,1,False,True,...,NaN,8512688.0,NaN,NaN,NaN,NaN,8513408.0,NaN,NaN,NaN
4,scaffold_1,9375101,9375387,Interval_298,-,g728,1,1,True,False,...,9375101,NaN,NaN,NaN,NaN,9375387,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,scaffold_142,12301,12875,Interval_9986,+,g22896,1,1,False,True,...,NaN,12301.0,NaN,NaN,NaN,NaN,12875.0,NaN,NaN,NaN
1917,scaffold_152,691,1979,Interval_9995,+,g146,1,1,False,True,...,NaN,691.0,NaN,NaN,NaN,NaN,1979.0,NaN,NaN,NaN
1918,scaffold_183,9853,10433,Interval_10010,+,g22851,1,1,False,True,...,NaN,9853.0,NaN,NaN,NaN,NaN,10433.0,NaN,NaN,NaN
1919,scaffold_263,7453,7925,Interval_27260,-,g22722,1,1,False,True,...,NaN,7453.0,NaN,NaN,NaN,NaN,7925.0,NaN,NaN,NaN


# Chi-squared statistical test (TO CORRECT)
## Step 1: Create a contingency table
### Males 
| MALE    | peak (DAR)    	                             | no peak (DAR)                                  |
|---------|---------------------------------------------|------------------------------------------------|
| DEG     | gene differentially expressed with peak     | gene differentially expressed without peak     |
| notDEG  | gene not differentially expressed with peak | gene not differentially expressed without peak |

-> Goal : answering the question "Is there a statistically significant association between the differential expression status of genes (DEG vs. not DEG) and the presence of a peak in differential accessible regions (DARs) within each sex?" 


In [17]:
# broad
DAR_broad_DEG_male_count = DAR_broad_DEG_male.shape[0]
notDAR_broad_DEG_male_count = DEG_male_tss_bed.shape[0] - DAR_broad_DEG_male.shape[0]
DAR_broad_notDEG_male_count = DAR_broad_TSS_male.shape[0] - DAR_broad_DEG_male.shape[0]
notDAR_broad_notDEG_male_count = all_expressed.shape[0] - DAR_broad_DEG_male_count - notDAR_broad_DEG_male_count - DAR_broad_notDEG_male_count

# contingency table for broad DAR male
table_broad_male = np.array([[DAR_broad_DEG_male_count, notDAR_broad_DEG_male_count],
                  [DAR_broad_notDEG_male_count, notDAR_broad_notDEG_male_count]])

table_broad_male

array([[ 329, 2538],
       [1592, 9649]])

In [18]:
# narrow
DAR_narrow_DEG_male_count = DAR_narrow_DEG_male.shape[0]
notDAR_narrow_DEG_male_count = DEG_male_tss_bed.shape[0] - DAR_narrow_DEG_male.shape[0]
DAR_narrow_notDEG_male_count = DAR_narrow_TSS_male.shape[0] - DAR_narrow_DEG_male.shape[0]
notDAR_narrow_notDEG_male_count = all_expressed.shape[0] - DAR_narrow_DEG_male_count - notDAR_narrow_DEG_male_count - DAR_narrow_notDEG_male_count

# contingency table for narrow DAR male
table_narrow_male = np.array([[DAR_narrow_DEG_male_count, notDAR_narrow_DEG_male_count],
                  [DAR_narrow_notDEG_male_count, notDAR_narrow_notDEG_male_count]])

table_narrow_male

array([[ 396, 2471],
       [2018, 9223]])

### Parthenos
One per sex, so for parthenos also:

| PARTHENO | peak (DAR)    	                              | no peak (DAR)                                  |
|----------|----------------------------------------------|------------------------------------------------|
| DEG      | gene differentially expressed with peak      | gene differentially expressed without peak     |
| notDEG   | gene not differentially expressed with peak  | gene not differentially expressed without peak |

In [19]:
# broad
DAR_broad_DEG_partheno_count = DAR_broad_DEG_partheno.shape[0]
notDAR_broad_DEG_partheno_count = DEG_partheno_tss_bed.shape[0] - DAR_broad_DEG_partheno.shape[0]
DAR_broad_notDEG_partheno_count = DAR_broad_TSS_partheno.shape[0] - DAR_broad_DEG_partheno.shape[0]
notDAR_broad_notDEG_partheno_count = all_expressed.shape[0] - DAR_broad_DEG_partheno_count - notDAR_broad_DEG_partheno_count - DAR_broad_notDEG_partheno_count

# contingency table for broad DAR partheno
table_broad_partheno = np.array([[DAR_broad_DEG_partheno_count, notDAR_broad_DEG_partheno_count],
                  [DAR_broad_notDEG_partheno_count, notDAR_broad_notDEG_partheno_count]])

table_broad_partheno

array([[   93,  1716],
       [  424, 11875]])

In [20]:
# narrow
DAR_narrow_DEG_partheno_count = DAR_narrow_DEG_partheno.shape[0]
notDAR_narrow_DEG_partheno_count = DEG_partheno_tss_bed.shape[0] - DAR_narrow_DEG_partheno.shape[0]
DAR_narrow_notDEG_partheno_count = DAR_narrow_TSS_partheno.shape[0] - DAR_narrow_DEG_partheno.shape[0]
notDAR_narrow_notDEG_partheno_count = all_expressed.shape[0] - DAR_narrow_DEG_partheno_count - notDAR_narrow_DEG_partheno_count - DAR_narrow_notDEG_partheno_count

# contingency table for narrow DAR partheno
table_narrow_partheno = np.array([[DAR_narrow_DEG_partheno_count, notDAR_narrow_DEG_partheno_count],
                  [DAR_narrow_notDEG_partheno_count, notDAR_narrow_notDEG_partheno_count]])

table_narrow_partheno

array([[  112,  1697],
       [  443, 11856]])

## Step 2 : Apply the chi-squared test on that crosstab

In [21]:
crosstabs = [
    (table_broad_male, 'Broad male'),
    (table_broad_partheno, 'Broad partheno'),
    (table_narrow_male, 'Narrow male'),
    (table_narrow_partheno, 'Narrow partheno')
]

for table, label in crosstabs:
    chi2, p_value, dof, expected = chi2_contingency(table)
    
    print(label)
    print('Chi-squared Statistic:', chi2)
    print('P-value:', p_value)
    print('Expected Frequencies:', expected)
    print('\n')

Broad male
Chi-squared Statistic: 13.794789454257891
P-value: 0.00020390094405709614
Expected Frequencies: [[ 390.38184009 2476.61815991]
 [1530.61815991 9710.38184009]]


Broad partheno
Chi-squared Statistic: 12.336755375664117
P-value: 0.0004441262815130554
Expected Frequencies: [[   66.2923873  1742.7076127]
 [  450.7076127 11848.2923873]]


Narrow male
Chi-squared Statistic: 27.311793822433486
P-value: 1.7315206713409603e-07
Expected Frequencies: [[ 490.56833003 2376.43166997]
 [1923.43166997 9317.56833003]]


Narrow partheno
Chi-squared Statistic: 27.29762965123209
P-value: 1.7442510286788888e-07
Expected Frequencies: [[   71.16494188  1737.83505812]
 [  483.83505812 11815.16494188]]




- Chi-squared statistic: measures the discrepancy between the observed and expected frequencies in the crosstab -> very low because our value are quite similar between observed and expected ?
- p-value not < 0.05: cannot reject the null hypothesis -> no statistically significant association between the expression and accessibility across male and parthenogenetic samples
- Expected Frequencies for the males: 44.8 for DAR_narrow_DEG_male_count and 2822.2 for the DEGs not DARs (the DEG_male_tss_bed that are not in the DEG + DARs list. For the parthenos: 28.2 and 1780.6.